In [1]:
!pip install nltk

In [2]:
import nltk

In [3]:
#nltk.download()

In [4]:
from nltk.corpus import wordnet
import os
import numpy as np
import pandas as pd

In [5]:
#directories
curr_dir = os.getcwd()
target_words_dir = 'Targets.txt'
target_txt = open(os.path.join(curr_dir,target_words_dir),"r")

def get_target_words(file_dir):
    target_txt = open(file_dir,"r")
    words = target_txt.readlines()
    return([word[:-1].lower() for word in words])

def get_dataframe(target_words):
    data = {"words":[],"description":[]}
    for word in target_words:
        try:
            synset = wordnet.synsets(word)         
            data["description"].append(synset[0].definition())
            data["words"].append(word)
        except:pass
    return pd.DataFrame(data)

In [6]:
Y = np.array(get_target_words(target_words_dir))
data = get_dataframe(Y)

In [8]:
data.head(12)

,words,description
0,kitty,the combined stakes of the betters
1,lookup,an operation that determines whether one or mo...
2,otiose,serving no useful purpose; having no excuse fo...
3,gallant,a man who is much concerned with his dress and...
4,sweetheart,a person loved by another person
5,run,a score in baseball made by a runner touching ...
6,monkish,befitting a monk; inclined to self-denial
7,orthodox,of or pertaining to or characteristic of Judaism
8,erosive,wearing away by friction
9,frivolity,the trait of being frivolous; not serious or s...
